# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [3]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [4]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [5]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [6]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [7]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [8]:
np.linalg.matrix_rank(a1)

6

In [10]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\penel\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\penel\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
C:\Users\penel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
C:\Users\penel\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [64]:
# ... ENTER YOUR CODE HERE ...
from numpy.linalg import *

leading_minor_a = np.zeros((N, N), dtype=float)
leading_minor_a1 = np.zeros((N, N), dtype=float)
for i in range(N):

    leading_minor_a[i,i] = det(a[:(i+1), :(i+1)])    #list the leading minors in a unit diagonal matrix
    if leading_minor_a[i,i] == 0:
        print("one of the leading minors equals to zero, thus the native LU decomposition cannot be succecced on this matrix a.\nthe zero position is %d * %d"%(i,i))
    leading_minor_a1[i,i] = det(a1[:(i+1), :(i+1)])
    if leading_minor_a1[i,i] == 0:
        print("one of the leading minors equals to zero, thus the native LU decomposition cannot be succecced on this matrix a1.\nthe zero position is %d * %d"%(i,i))
    
print("leading minors for a: \n", leading_minor_a, "\n")
print("leading minors for a1: \n", leading_minor_a1, "\n")


one of the leading minors equals to zero, thus the native LU decomposition cannot be succecced on this matrix a1.
the zero position is 1 * 1
leading minors for a: 
 [[ 3.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -3.375e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -8.860e-01  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  1.947e-02  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.573e-05  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -2.492e-10]] 

leading minors for a1: 
 [[ 3.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -8.033e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -4.935e-01  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  9.030e-04  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [39]:
# ... ENTER YOUR CODE HERE ...
def diy_lu_colPivot(a):

    N = a.shape[0]  
    u = a.copy()
    L = np.eye(N)   
    P = np.eye(N) 
    for j in range(N-1):  
        col_max = np.max(np.abs(u[j:,j]))                                   #find the max number in one column
        col_max_row = list(np.where(np.abs(u[j:,j])==col_max))[0][0] + j    #find the position of the max number in the column
 
        col_pivoting = False
        if col_max_row != j:
            P[[col_max_row, j], :] = P[[j, col_max_row], :]       # build permutation matrice to track the row swapping
            u[[col_max_row, j], :] = u[[j, col_max_row], :]       #finish swapping rows
            col_pivoting = True
            
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]   
        lam[j+1:, j] = -gamma
        u = lam @ u 
        lam[j+1:, j] = gamma
        if col_pivoting == True:
            L[[col_max_row, j], :j] = L[[j, col_max_row], :j]
        L = L @ lam
    return L, u, P


In [70]:
L, U, P = diy_lu_colPivot(a)

print("L: \n", L, "\n")
print("U: \n", U, "\n")
print("P: \n", P, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@U - P@a: \n", L@U - P@a, "\n")
if np.max(L@U - P@a) < 1.0e-15:
    print("the LU pivoting result is correct.")

L: 
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]] 

U: 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  1.110e-16 -2.317e-17 -1.999e-19 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.431e-16  6.463e-18  1.577e-19  0.000e+00  6.730e-06]] 

P: 
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]] 

L@U - P@a: 
 [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16  0.00

In [71]:
L1, U1, P1 = diy_lu_colPivot(a1)

print("L1: \n", L1, "\n")
print("U1: \n", U1, "\n")
print("P1: \n", P1, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L1@U1 - P1@a1: \n", L1@U1 - P1@a1, "\n")
if np.max(L1@U1 - P1@a1) < 1.0e-15:
    print("the LU pivoting result is correct.")

L1: 
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0.088 0.514 1.    0.   ]
 [1.    0.941 0.038 0.208 0.641 1.   ]] 

U1: 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.776e-17 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  1.110e-16 -1.427e-17 -4.160e-18  1.841e-03  3.821e-03]
 [ 0.000e+00 -1.267e-16  3.391e-18  2.669e-18  0.000e+00 -1.233e-05]] 

P1: 
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]] 

L1@U1 - P1@a1: 
 [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+0

In [74]:
def reconstruct_original(L, U, P):
    
    a = L @ U
    A = np.transpose(P) @ a
    
    return A

a_reconstruct = reconstruct_original(L, U, P)
a1_reconstruct = reconstruct_original(L1, U1, P1)

print(a_reconstruct - a, "\n")
if np.max(a_reconstruct - a) < 1.0e-15:
    print("the reconstruction result of a is correct.\n")
print(a1_reconstruct - a1, "\n")
if np.max(a1_reconstruct - a1) < 1.0e-15:
    print("the reconstruction result of a1 is correct.\n")

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17  2.776e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16  5.551e-17]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]] 

the reconstruction result of a is correct.

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16  4.996e-16]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]] 

the reconstruction result of a1 is correct.

